In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# CHECK WORKIND DIR PATH
import os
# MODIFY SYSTEM PATH
os.chdir('../')
os.getcwd()

'/home/christophe/workspace/arreason/CategoryLearning'

In [3]:
from typing import Callable, Iterable, Any
import sys
import torch
import random
import warnings
# Needed to show warnings in all Jupyter distributions (e.g. VS Code Jupyter implementation)
warnings.simplefilter(action="default")
from tqdm import tnrange
sys.path.append('../catlearn')
from catlearn.data.dataset import Dataset
from catlearn.tensor_utils import (Tsor, DEFAULT_EPSILON)
from catlearn.graph_utils import (DirectedGraph, uniform_sample, random_walk_edge_sample,
                                    random_walk_vertex_sample)
from catlearn.algebra_models import (Algebra, VectAlgebra, VectMultAlgebra)
from catlearn.composition_graph import CompositeArrow
from catlearn.categorical_model import (TrainableDecisionCatModel, RelationModel,
                                        ScoringModel)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [4]:
from wandb_logger import log_results

/home/christophe/workspace/arreason/CategoryLearning/Python/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/christophe/workspace/arreason/CategoryLearning/Python/lib/python3.8/site-packages/wandb/util.py:36: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import namedtuple, Mapping, Sequence
/home/christophe/workspace/arreason/CategoryLearning/Python/lib/python3.8/site-packages/wandb/vendor/graphql-core-1.1/graphql/type/directives.py:55: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since 

In [5]:
# DEBUG: Tools to export networkx graph
from networkx import to_dict_of_lists, to_dict_of_dicts

/home/christophe/workspace/arreason/CategoryLearning/Python/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# Select default type: https://pytorch.org/docs/stable/tensors.html
default_tensor = torch.FloatTensor
# default_tensor = torch.cuda.FloatTensor
torch.set_default_tensor_type(default_tensor)
# Check default tensor datatype
default_tensor.dtype

torch.float32

In [7]:
class CustomRelation(RelationModel):
    """ Fake relations
    two nodes and rel
    """
    def __init__(self, nb_features: int, nb_labels: int, algebra: Algebra) -> None:
        self.linear = torch.nn.Linear(2 * nb_features + nb_labels, algebra.flatdim)

    @property
    def parameters(self) -> Callable[[], Iterable[Any]]:
        return self.linear.parameters

    def __call__(self, x: Tsor, y: Tsor, l: Tsor) -> Tsor:
        """ Compute x R y """
        return self.linear(torch.cat((x, y, l), -1))

In [8]:
class CustomScore(ScoringModel):
    """ Must be defined. depends on algebra and 
    in the scope of definition of the project. """
    def __init__(
            self,
            nb_features: int,
            nb_scores: int,
            algebra: Algebra) -> None:
        self.linear = torch.nn.Linear(
            2 * nb_features + algebra.flatdim, nb_scores + 1)
        self.softmax = torch.nn.Softmax(dim=-1)

    @property
    def parameters(self) -> Callable[[], Iterable[Any]]:
        return self.linear.parameters

    def __call__(self, src: Tsor, dst: Tsor, rel: Tsor) -> Tsor:
        """ Compute S(src, dst, rel) """
        cat_input = torch.cat((src, dst, rel), -1)
        return self.softmax(self.linear(cat_input))[..., :-1]

In [9]:
# Specify dataset path
ds_path_wn18 = './Datasets/wn18rr/text'
ds_path_fb15 = './Datasets/fb15k-237'

In [10]:
ds_wn18 = Dataset(path=ds_path_wn18, ds_name='wn18', node_vec_dim=10)

/home/christophe/workspace/arreason/CategoryLearning/catlearn/data/dataset.py:123: ResourceWarning: unclosed file <_io.TextIOWrapper name='Datasets/wn18rr/text/entity_to_id.txt' mode='r' encoding='UTF-8'>
  return {e: int(id) for e, id in id_map}
/home/christophe/workspace/arreason/CategoryLearning/catlearn/data/dataset.py:123: ResourceWarning: unclosed file <_io.TextIOWrapper name='Datasets/wn18rr/text/relation_to_id.txt' mode='r' encoding='UTF-8'>
  return {e: int(id) for e, id in id_map}


In [11]:
# ds_fb15 = Dataset(path=ds_path_fb15, ds_name='fb15', node_vec_dim=10)

In [12]:
# MODIFY BELOW TO USE RIGHT DATASET
ds = ds_wn18

In [13]:
# DEBUG: convert dataset to a list
# ds_l = list(ds.train)

In [14]:
algebra = VectMultAlgebra(ds.entity_vec_dim)

In [15]:
relation_model = CustomRelation(
    nb_features=ds.entity_vec_dim,
    nb_labels=len(ds.relation2id),
    algebra=algebra
)

In [16]:
scoring_model = CustomScore(
    nb_features=ds.entity_vec_dim,
    nb_scores=len(ds.relation2id),
    algebra=algebra
)

In [17]:
model = TrainableDecisionCatModel(
    relation_model=relation_model,
    label_universe=ds.relation_id2vec,
    scoring_model=scoring_model,
    algebra_model=algebra,
    optimizer=torch.optim.Adam,
    epsilon=DEFAULT_EPSILON
)

In [18]:
# DEBUG: Represent graph as dict of dicts or dict of lists
# dod = to_dict_of_dicts(labels)
# dod[10698]
# dol = to_dict_of_lists(labels)
# dol[10698]

In [19]:
# DEBUG: check OHE relations encoding
ds.relation_id2vec

{0: tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 1: tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 2: tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 3: tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 4: tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 5: tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 6: tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 7: tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 8: tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 9: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 10: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}

In [20]:
# DEBUG: check relations to id conversion dict
ds.relation2id

{'_instance_hypernym': 0,
 '_hypernym': 1,
 '_derivationally_related_form': 2,
 '_synset_domain_topic_of': 3,
 '_similar_to': 4,
 '_member_meronym': 5,
 '_has_part': 6,
 '_member_of_domain_usage': 7,
 '_verb_group': 8,
 '_also_see': 9,
 '_member_of_domain_region': 10}

In [21]:
# DEBUG NOTE: datatype comparaison
# Dataset interable format [src:int, tgt:int, lbl: {id:int: vec:Tsor}]
# CompositeArrow data format:  [[src: int, tgt: int], [label: int]]

In [22]:
graph_labels = DirectedGraph(ds.train)

/home/christophe/workspace/arreason/CategoryLearning/catlearn/data/dataset.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='Datasets/wn18rr/text/train.txt' mode='r' encoding='UTF-8'>
  return ((


In [23]:
def graph_to_composite_arrow(graph: DirectedGraph):
    nodes = [(src, dst) for src, dst, _ in graph.edges(data=True)]
    edges = [list(rel.keys()) for _, _, rel in graph.edges(data=True)]
    return nodes, edges

/home/christophe/workspace/arreason/CategoryLearning/Python/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
# NOTE: for large graphs, random_walk functions family can be used to sub-sample graph
# while preserving its topology 
for i in tnrange(10, desc='1st loop: batches'):
    arrows_graph = uniform_sample(graph=graph_labels, sample_vertices_size=7, rng=random.Random(), with_edges=True)
    nodes, edges = graph_to_composite_arrow(arrows_graph)
    if edges:
        arrows = [CompositeArrow(nodes=node, arrows=edge) for node, edge in zip(nodes, edges)]
    else:
        arrows = [CompositeArrow(nodes=node) for node in nodes]
    cache, matches = model.train(
        data_points = ds.entity_id2vec,
        relations = arrows,
        # NOTE: Labels could be a couplete graph, a subgraph from random_walk or a sub-sub-graph used to create a batch
        labels = arrows_graph,
        step = True,
        match_negatives=False
    )
    log_results(cache, matches)

<ipython-input-24-8d852144d3e6>:3: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(10, desc='1st loop: batches'):


/home/christophe/workspace/arreason/CategoryLearning/catlearn/graph_utils.py:632: UserWarning: sample_vertices_size 7 is odd and will be rounded to floor even number 6when with_edges=True
  warnings.warn(str_color('W', warning_str), UserWarning)

